In [5]:
import pandas as pd
import json
import time
import resource
import numpy as np

from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_qdrant import Qdrant

import qdrant_client
from qdrant_client import QdrantClient
from qdrant_client.models import (
    PointStruct, 
    VectorParams, 
    Distance, 
    SparseVector, 
    SparseVectorParams,
    SparseIndexParams,
    NamedVector,
    NamedSparseVector,
    SearchRequest
)

from sentence_transformers import SentenceTransformer, CrossEncoder
from fastembed import SparseTextEmbedding, TextEmbedding


In [ ]:
# Configuration
url = "http://localhost:6333"
collection_name = "faq-hybrid"
distance = Distance.COSINE
dense_dimension = 1024

# Initialize models
# Dense embeddings model
dense_embeddings = SentenceTransformer(
    "jinaai/jina-embeddings-v3",
    trust_remote_code=True
)

# Sparse embeddings model (new addition)
# Using a multilingual SPLADE model for Indonesian
sparse_model = SentenceTransformer(
    "intfloat/multilingual-e5-large", 
    trust_remote_code=True
)


No sentence-transformers model found with name intfloat/multilingual-e5-large. Creating a new one with mean pooling.


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like intfloat/multilingual-e5-large is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [3]:
SparseTextEmbedding.list_supported_models()

[{'model': 'prithivida/Splade_PP_en_v1',
  'sources': {'hf': 'Qdrant/SPLADE_PP_en_v1', 'url': None},
  'model_file': 'model.onnx',
  'description': 'Independent Implementation of SPLADE++ Model for English.',
  'license': 'apache-2.0',
  'size_in_GB': 0.532,
  'additional_files': [],
  'requires_idf': None,
  'vocab_size': 30522},
 {'model': 'prithvida/Splade_PP_en_v1',
  'sources': {'hf': 'Qdrant/SPLADE_PP_en_v1', 'url': None},
  'model_file': 'model.onnx',
  'description': 'Independent Implementation of SPLADE++ Model for English.',
  'license': 'apache-2.0',
  'size_in_GB': 0.532,
  'additional_files': [],
  'requires_idf': None,
  'vocab_size': 30522},
 {'model': 'Qdrant/bm42-all-minilm-l6-v2-attentions',
  'sources': {'hf': 'Qdrant/all_miniLM_L6_v2_with_attentions', 'url': None},
  'model_file': 'model.onnx',
  'description': 'Light sparse embedding model, which assigns an importance score to each token in the text',
  'license': 'apache-2.0',
  'size_in_GB': 0.09,
  'additional_f